In [1]:
input_dim = 4
num_clusters = 4

ntrials = 10
lg = 1.5 #lambda_global

import numpy as np
import os
from sklearn.cluster import KMeans

import sys
sys.path.insert(0, "../Code/")
from myplot import plot_groups, plot_metrics
from train_multiclass import train_multiclass

old_stdout = sys.stdout

In [3]:
# Configure the data generation process and train the autoencoder

def gen(n = 400):
    
    x = np.zeros((n, 4))
    y = np.zeros((n))
    
    for i in range(n):
        
        if np.random.uniform() < 0.5:
            x[i, 0] = 1.0 + np.random.normal(loc = 0.0, scale = 0.2)
            y[i] += 1
            
        if np.random.uniform() < 0.5:
            x[i, 1] = 1.0 + np.random.normal(loc = 0.0, scale = 0.2)
            y[i] += 2
            
        x[i, 2] = np.random.normal(loc = 0.0, scale = 0.5)
        
        x[i, 3] = x[i, 0] + np.random.normal(loc = 0.0, scale = 0.05)
        
    return x, y

x, y = gen()

y = y.astype(int)

os.system("rm -rf Model")

sess, rep, prob, grad_i, X, D, Y, I = train_multiclass(x, y, 4)

sys.stdout = old_stdout


INFO:tensorflow:Summary name Cross Entropy is illegal; using Cross_Entropy instead.
INFO:tensorflow:Summary name Recon MSE is illegal; using Recon_MSE instead.
INFO:tensorflow:Restoring parameters from ./model.cpkt


In [4]:
indices = []
for i in range(num_clusters):
    v = []
    for j in range(x.shape[0]):
        if y[j] == i:
            v.append(j)
    indices.append(v)


In [5]:
def info(x, i):
    p, g = sess.run([prob, grad_i], feed_dict = {X: x, I: i, D: np.zeros((1, input_dim))})
    return p, g[0]

sys.path.insert(0, "../Integrated-Gradients-master/IntegratedGradients/")
from integrated_gradients import integrated_gradients

def ig(init, target, avg_class = False, num_classes = num_clusters, normalize = True):
    average = np.zeros((input_dim))
    count = 0
    for index_ref in indices[init]:
        baseline = x[index_ref, :]
        for index_target in indices[target]:
            point = x[index_target, :]
            if avg_class:
                for target_class in range(num_classes):
                    ig, _ = integrated_gradients(point, target_class, info, baseline)
                    average += ig
                    count += 1
            else:
                ig, _ = integrated_gradients(point, target, info, baseline)
                average += ig
                count += 1
    average /= count
    if normalize:
        average /= np.sum(np.abs(average))
        average = np.round(average, decimals = 3)        
    return average 

deltas = []
for g in range(1, num_clusters):
    deltas.append(ig(0, g))
    
deltas = np.array(deltas)
np.save("deltas_ig.npy", deltas)


print("IG consistently uses the SC variable")
print(deltas)


IG consistently uses the SC variable
[[ 0.795  0.     0.009  0.196]
 [ 0.     0.998  0.002 -0.   ]
 [ 0.56   0.273  0.004  0.163]]


In [6]:
print("IG is not always symmetrical")

print(ig(1, 2))
print(ig(2, 1))

print(ig(1, 2, avg_class = True, normalize = False))
print(ig(2, 1, avg_class = True, normalize = False))



IG is not always symmetrical
[ 0.701 -0.088  0.     0.211]
[ 0.131  0.787 -0.01   0.073]
[-5.14206574e-09  2.30849037e-08 -7.12789813e-10  7.21088134e-10]
[ 9.17419479e-09 -1.48178121e-08  7.19630824e-10 -2.41104483e-10]


In [7]:

i1 = indices[1][3]
i2 = indices[2][0]

x1 = x[i1, :]
t1 = y[i1]
x2 = x[i2, :]
t2 = y[i2]

print("Class 1 ", t1,  " Class 2 ", t2)

print("IG is symmetrical when the `class of interest' is held constant")

print("Setting it to ", t1)
v, _ = integrated_gradients(x1, t1, info, x2)
print(v)
v, _ = integrated_gradients(x2, t1, info, x1)
print(v)
print("Setting it to ", t2)
v, _ = integrated_gradients(x1, t2, info, x2)
print(v)
v, _ = integrated_gradients(x2, t2, info, x1)
print(v)
print("But it isn't as soon as you ask `why is this point in its assigned class as opposed to that other point?'")

o = 0
for i in range(num_clusters):
    print("Target ", i)
    v, _ = integrated_gradients(x1, i, info, x2)
    print(v)
    o += v
o /= num_clusters
print("Averaged: ", o)

o = 0
for i in range(num_clusters):
    print("Target ", i)
    v, _ = integrated_gradients(x2, i, info, x1)
    print(v)
    o += v
o /= num_clusters
print("Averaged: ", o)


Class 1  1  Class 2  2
IG is symmetrical when the `class of interest' is held constant
Setting it to  1
[ 0.15145647  1.47499919 -0.05085756  0.08864364]
[-0.15145647 -1.47499935  0.05085756 -0.08864364]
Setting it to  2
[-0.81982442  0.05155611 -0.03824885 -0.20114954]
[ 0.81982463 -0.05155611  0.03824887  0.20114951]
But it isn't as soon as you ask `why is this point in its assigned class as opposed to that other point?'
Target  0
[-4.55003098e-04 -2.48742502e-03  5.26528866e-05 -3.08719855e-04]
Target  1
[ 0.15145647  1.47499919 -0.05085756  0.08864364]
Target  2
[-0.81982442  0.05155611 -0.03824885 -0.20114954]
Target  3
[ 0.66882315 -1.52406865  0.08905381  0.11281463]
Averaged:  [ 4.89437084e-08 -1.94187014e-07  1.11358205e-08  2.41314406e-09]
Target  0
[ 4.55003132e-04  2.48742565e-03 -5.26528923e-05  3.08719855e-04]
Target  1
[-0.15145647 -1.47499935  0.05085756 -0.08864364]
Target  2
[ 0.81982463 -0.05155611  0.03824887  0.20114951]
Target  3
[-0.668823    1.52406849 -0.089053